# Imports

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
con = sqlite3.connect('/Users/kellyjara/Desktop/Project1/Data/im.db')

# Creating a table with actors/actresses and their name

In [3]:
#loading and specifying columns in principals table
persons_ = pd.read_sql("""
SELECT movie_id, person_id, category
FROM principals
WHERE category = 'actor' OR category = 'actress';
""", con)

In [4]:
#loading and specifying columns in persons table
persons_name = pd.read_sql("""
SELECT person_id, primary_name, primary_profession
FROM persons
WHERE primary_profession = 'actor' OR primary_profession = 'actress'
""", con)

In [5]:
#converting tables to PD DataFrame
persons_df = pd.DataFrame(persons_)
persons_name_df = pd.DataFrame(persons_name)

In [6]:
#merging both DataFrames based on common column: person_id
merged_df = pd.merge(persons_, persons_name, on = 'person_id')
merged_df

,movie_id,person_id,category,primary_name,primary_profession
0,tt0111414,nm0246005,actor,Tommy Dysart,actor
1,tt0323808,nm2694680,actor,Henry Garrett,actor
2,tt0323808,nm0502652,actress,Jacqueline Leonard,actress
3,tt0879405,nm0442635,actress,Ingrid Kavelaars,actress
4,tt0879405,nm0516665,actress,Hannah Lochner,actress
...,...,...,...,...,...
207440,tt9681728,nm10436664,actor,Filipp Dyachkov,actor
207441,tt9681728,nm8254972,actor,Sergey Goncharenko,actor
207442,tt9681728,nm10436662,actor,Vladimir Korolev,actor
207443,tt9681728,nm10436663,actor,Artem Leshik,actor


In [7]:
merged_df.drop(columns = 'primary_profession', inplace = True)

# Adding movie_basics table & ratings table to the DF

In [8]:
#adding movie basics to a dataframe
movie_info = pd.read_sql("""
SELECT * FROM movie_basics;
""", con)

In [9]:
movies = pd.DataFrame(movie_info)

In [10]:
movies.dropna(axis = 0, inplace = True)

In [11]:
#merging movie dataframe 
movies_actors = pd.merge(merged_df, movies, on = 'movie_id')

In [12]:
#adding movie ratings to a dataframe
ratings = pd.read_sql("""
SELECT * FROM movie_ratings
""", con)

In [13]:
movie_ratings = pd.DataFrame(ratings)

In [14]:
#merging dataframes w. dataframe that was made with actors/actresses info
ratings_mov = pd.merge(movies_actors, movie_ratings, on = 'movie_id')
ratings_mov

,movie_id,person_id,category,primary_name,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0323808,nm2694680,actor,Henry Garrett,The Wicker Tree,The Wicker Tree,2011,96.0,"Drama,Horror",3.9,2328
1,tt0323808,nm0502652,actress,Jacqueline Leonard,The Wicker Tree,The Wicker Tree,2011,96.0,"Drama,Horror",3.9,2328
2,tt0879405,nm0442635,actress,Ingrid Kavelaars,Harm's Way,Harm's Way,2010,82.0,Thriller,5.3,250
3,tt0879405,nm0516665,actress,Hannah Lochner,Harm's Way,Harm's Way,2010,82.0,Thriller,5.3,250
4,tt0972848,nm2741095,actor,Errol Sperling,Lives and Deaths of the Poets,Lives and Deaths of the Poets,2011,93.0,Comedy,7.7,6
...,...,...,...,...,...,...,...,...,...,...,...
105483,tt9681728,nm10436664,actor,Filipp Dyachkov,A Russian Youth,Malchik russkiy,2019,72.0,"Drama,War",6.6,34
105484,tt9681728,nm8254972,actor,Sergey Goncharenko,A Russian Youth,Malchik russkiy,2019,72.0,"Drama,War",6.6,34
105485,tt9681728,nm10436662,actor,Vladimir Korolev,A Russian Youth,Malchik russkiy,2019,72.0,"Drama,War",6.6,34
105486,tt9681728,nm10436663,actor,Artem Leshik,A Russian Youth,Malchik russkiy,2019,72.0,"Drama,War",6.6,34


# Adding movie gross values to table

In [15]:
tn_movie = pd.read_csv("/Users/kellyjara/Desktop/Project1/Data/tn.movie_budgets.csv")
tn_movie

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


# Merging all tables

In [16]:
movie_full = pd.merge(ratings_mov, tn_movie, left_on = 'primary_title', right_on = 'movie')
movie_full

,movie_id,person_id,category,primary_name,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,tt1083452,nm8030441,actor,Tom Costello,Eddie the Eagle,Eddie the Eagle,2015,106.0,"Biography,Comedy,Drama",7.4,75331,86,"Feb 26, 2016",Eddie the Eagle,"$23,000,000","$15,789,389","$45,060,480"
1,tt1124037,nm0991810,actor,Mahershala Ali,Free State of Jones,Free State of Jones,2016,139.0,"Action,Biography,Drama",6.9,45135,81,"Jun 24, 2016",Free State of Jones,"$50,000,000","$20,810,036","$23,237,252"
2,tt0437086,nm0991810,actor,Mahershala Ali,Alita: Battle Angel,Alita: Battle Angel,2019,122.0,"Action,Adventure,Sci-Fi",7.5,88207,24,"Feb 14, 2019",Alita: Battle Angel,"$170,000,000","$85,710,210","$402,976,036"
3,tt0437086,nm0000124,actress,Jennifer Connelly,Alita: Battle Angel,Alita: Battle Angel,2019,122.0,"Action,Adventure,Sci-Fi",7.5,88207,24,"Feb 14, 2019",Alita: Battle Angel,"$170,000,000","$85,710,210","$402,976,036"
4,tt4975722,nm0991810,actor,Mahershala Ali,Moonlight,Moonlight,2016,111.0,Drama,7.4,227964,64,"Oct 21, 2016",Moonlight,"$1,500,000","$27,854,931","$65,245,512"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2868,tt7535280,nm9358590,actor,Andrei Zagorodnii,Teeth,Dintii,2017,71.0,Drama,8.8,25,48,"Jan 18, 2008",Teeth,"$2,000,000","$347,578","$2,350,641"
2869,tt8662424,nm9953795,actress,Toni Allan,Never Again,Never Again,2017,106.0,Drama,5.7,67,47,"Jul 12, 2002",Never Again,"$500,000","$307,631","$308,793"
2870,tt8662424,nm9953794,actress,Kelly Baxter,Never Again,Never Again,2017,106.0,Drama,5.7,67,47,"Jul 12, 2002",Never Again,"$500,000","$307,631","$308,793"
2871,tt8662424,nm9953791,actor,Hawt Carl,Never Again,Never Again,2017,106.0,Drama,5.7,67,47,"Jul 12, 2002",Never Again,"$500,000","$307,631","$308,793"


# Creating a new column for ROI 

In [17]:
#converting domestic gross into an int type
movie_full['domestic_gross'] = movie_full['domestic_gross'].str.replace('$',"").str.replace(',',"").astype(int)

In [18]:
#converting worldwide gross into an int type
movie_full['worldwide_gross'] = movie_full['worldwide_gross'].str.replace('$',"").str.replace(',',"").astype('int64')

In [19]:
movie_full['production_budget'] = movie_full['production_budget'].str.replace('$',"").str.replace(',',"").astype(int)

In [20]:
movie_full['ROI'] = (movie_full['worldwide_gross'] - movie_full['production_budget'])/movie_full['production_budget']

# OBTAINING VALUES FOR PLOTS

In [21]:
high_rating = movie_full.loc[movie_full['averagerating'] > 7.0]

In [22]:
low_rating = movie_full.loc[movie_full['averagerating'] < 3.0]

# ...ACTOR INFO...

In [28]:
actor = movie_full[(movie_full['category'] == 'actor') & (movie_full['averagerating'] >= 7.0)]
actor.sort_values(by = 'averagerating', ascending = False, inplace = False)

,movie_id,person_id,category,primary_name,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,ROI
1050,tt4061640,nm2584297,actor,Alan Brent,Traffic,Traffic,2014,110.0,Documentary,9.2,64,16,"Dec 27, 2000",Traffic,48000000,124107476,208300000,3.339583
1872,tt6168914,nm0090196,actor,Lee Boardman,The Runaways,The Runaways,2019,108.0,Adventure,9.2,47,57,"Mar 19, 2010",The Runaways,9500000,3573673,5278632,-0.444355
2290,tt1795369,nm4597694,actor,John Killoran,Frankenstein,National Theatre Live: Frankenstein,2011,130.0,Drama,9.0,1832,3,"Nov 4, 1994",Frankenstein,45000000,22006296,112006296,1.489029
2450,tt3060772,nm5806940,actor,Alison Kam,Dragonfly,Dragonfly,2013,75.0,"Action,Adventure,Sci-Fi",9.0,5,28,"Feb 22, 2002",Dragonfly,60000000,30063805,30063805,-0.498937
2448,tt3060772,nm5806936,actor,Mitchell Campos,Dragonfly,Dragonfly,2013,75.0,"Action,Adventure,Sci-Fi",9.0,5,28,"Feb 22, 2002",Dragonfly,60000000,30063805,30063805,-0.498937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2656,tt3721936,nm6543383,actor,McCaul Lombardi,American Honey,American Honey,2016,163.0,"Adventure,Drama,Romance",7.0,31663,15,"Sep 30, 2016",American Honey,3500000,663247,2611750,-0.253786
1074,tt1828325,nm0890632,actor,Andi Vasluianu,The Gambler,The Gambler,2015,105.0,Drama,7.0,10,25,"Dec 10, 2014",The Gambler,25000000,33680992,38718031,0.548721
1489,tt1259528,nm6578009,actor,O'Shea Jackson Jr.,Den of Thieves,Den of Thieves,2018,140.0,"Action,Crime,Drama",7.0,69648,3,"Jan 19, 2018",Den of Thieves,30000000,44947622,80131391,1.671046
1883,tt1615160,nm0849214,actor,Mark Tandy,The Foreigner,The Foreigner,2017,113.0,"Action,Thriller",7.0,81288,81,"Oct 13, 2017",The Foreigner,35000000,34393507,140783360,3.022382


In [29]:
actor = movie_full[(movie_full['category'] == 'actor')]
actor.sort_values(by = 'averagerating', ascending = False, inplace = False)

,movie_id,person_id,category,primary_name,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,ROI
1872,tt6168914,nm0090196,actor,Lee Boardman,The Runaways,The Runaways,2019,108.0,Adventure,9.2,47,57,"Mar 19, 2010",The Runaways,9500000,3573673,5278632,-0.444355
1050,tt4061640,nm2584297,actor,Alan Brent,Traffic,Traffic,2014,110.0,Documentary,9.2,64,16,"Dec 27, 2000",Traffic,48000000,124107476,208300000,3.339583
2448,tt3060772,nm5806936,actor,Mitchell Campos,Dragonfly,Dragonfly,2013,75.0,"Action,Adventure,Sci-Fi",9.0,5,28,"Feb 22, 2002",Dragonfly,60000000,30063805,30063805,-0.498937
2290,tt1795369,nm4597694,actor,John Killoran,Frankenstein,National Theatre Live: Frankenstein,2011,130.0,Drama,9.0,1832,3,"Nov 4, 1994",Frankenstein,45000000,22006296,112006296,1.489029
2450,tt3060772,nm5806940,actor,Alison Kam,Dragonfly,Dragonfly,2013,75.0,"Action,Adventure,Sci-Fi",9.0,5,28,"Feb 22, 2002",Dragonfly,60000000,30063805,30063805,-0.498937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,tt3309662,nm6089600,actor,Bharath Niwas,Jackpot,Jackpot,2013,132.0,"Comedy,Thriller",2.1,647,17,"Jul 27, 2001",Jackpot,400000,44452,44452,-0.888870
2292,tt2286560,nm5175755,actor,Chris Party,Frankenstein,Frankenstein,2011,82.0,"Drama,Horror",1.9,67,3,"Nov 4, 1994",Frankenstein,45000000,22006296,112006296,1.489029
2293,tt2286560,nm5175669,actor,Dean Gangle,Frankenstein,Frankenstein,2011,82.0,"Drama,Horror",1.9,67,3,"Nov 4, 1994",Frankenstein,45000000,22006296,112006296,1.489029
2294,tt2286560,nm2416968,actor,Haydon Downing,Frankenstein,Frankenstein,2011,82.0,"Drama,Horror",1.9,67,3,"Nov 4, 1994",Frankenstein,45000000,22006296,112006296,1.489029


In [25]:
actor_counts = actor['primary_name'].value_counts().sort_values(ascending = False, inplace = False).head(20)
actor_counts

Mark Strong           11
Dane DeHaan            8
Richard Jenkins        8
Armie Hammer           7
Luke Bracey            5
Jacob Tremblay         5
Sebastian Stan         5
Matthew Macfadyen      4
Henry Cavill           4
Joel Kinnaman          4
David Harbour          4
Mahershala Ali         4
Dylan O'Brien          4
Kodi Smit-McPhee       4
Karl Urban             4
Ilhami Sibil           3
Alfie Allen            3
Prabhas                3
Muhammet Uzuner        3
O'Shea Jackson Jr.     3
Name: primary_name, dtype: int64

# ACTRESS INFO

In [26]:
actress = movie_full[(movie_full['category'] == 'actress') & (movie_full['averagerating'] >= 7.0)]
actress.sort_values(by = 'averagerating', ascending = False, inplace = False)

,movie_id,person_id,category,primary_name,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,ROI
1873,tt6168914,nm2703558,actress,Molly Windsor,The Runaways,The Runaways,2019,108.0,Adventure,9.2,47,57,"Mar 19, 2010",The Runaways,9500000,3573673,5278632,-0.444355
2291,tt1795369,nm2132319,actress,Andreea Paduraru,Frankenstein,National Theatre Live: Frankenstein,2011,130.0,Drama,9.0,1832,3,"Nov 4, 1994",Frankenstein,45000000,22006296,112006296,1.489029
1669,tt8439948,nm8832992,actress,Rashmi Bhatta,Bobby,Bobby,2018,130.0,Drama,8.9,23,8,"Nov 17, 2006",Bobby,14000000,11242801,20597806,0.471272
1670,tt8439948,nm9851378,actress,Kabita Gurung Thapa,Bobby,Bobby,2018,130.0,Drama,8.9,23,8,"Nov 17, 2006",Bobby,14000000,11242801,20597806,0.471272
2317,tt8259618,nm9762527,actress,Mawar Eva de Jongh,Serendipity,Serendipity,2018,107.0,Drama,8.8,53,53,"Oct 5, 2001",Serendipity,28000000,50255310,77477297,1.767046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2041,tt1945037,nm2472972,actress,Savanna Kylie Lewis,Begin Again,Begin Again,2013,68.0,"Drama,Family",7.0,57,91,"Jun 27, 2014",Begin Again,8000000,16170632,68838736,7.604842
1678,tt3104988,nm2110418,actress,Gemma Chan,Crazy Rich Asians,Crazy Rich Asians,2018,120.0,"Comedy,Romance",7.0,96617,38,"Aug 15, 2018",Crazy Rich Asians,30000000,174532921,238099711,6.936657
465,tt6682904,nm0586712,actress,Egle Mikulionyte,Miracle,Stebuklas,2017,91.0,"Comedy,Drama",7.0,420,45,"Feb 6, 2004",Miracle,28000000,64378093,64474705,1.302668
1301,tt1259521,nm0404307,actress,Anna Hutchison,The Cabin in the Woods,The Cabin in the Woods,2011,95.0,Horror,7.0,338445,11,"Apr 13, 2012",The Cabin in the Woods,30000000,42073277,70689968,1.356332


In [27]:
actress_counts = actress['primary_name'].value_counts().sort_values(ascending = False, inplace = False).head(20)